In [2]:
link = 'https://raw.githubusercontent.com/ricardorocha86/Datasets/master/PesquisaDH/datahackers-survey-2019-anonymous-responses.csv'
import pandas as pd
dados = pd.read_csv(link)
dados.head()

,"('P0', 'id')","('P1', 'age')","('P2', 'gender')","('P3', 'living_in_brasil')","('P5', 'living_state')","('P6', 'born_or_graduated')","('P8', 'degreee_level')","('P10', 'job_situation')","('P12', 'workers_number')","('P13', 'manager')","('P16', 'salary_range')","('P17', 'time_experience_data_science')","('P18', 'time_experience_before')","('P19', 'is_data_science_professional')","('P20', 'linear_regression')","('P20', 'logistic_regression')","('P20', 'glms')","('P20', 'decision_tree')","('P20', 'random_forest')","('P20', 'neural_networks')","('P20', 'bayesian_inference')","('P20', 'ensemble')","('P20', 'svms')","('P20', 'cnns')","('P20', 'rnns')","('P20', 'hmms')","('P20', 'gans')","('P20', 'markov_chains')","('P20', 'nlp')","('P20', 'gradient_boosted_machines')","('P20', 'cluster_analysis')","('P20', 'survival_analysis')","('P20', 'longitudinal_data_analysis')","('P20', 'joint analysis')","('P20', 'no_listed_methods')","('P21', 'sql_')","('P21', 'r')","('P21', 'python')","('P21', 'c_c++_c#')","('P21', 'dotnet')",...,"('P30', 'microsoft_azure')","('P30', 'do_not_know')","('P30', 'other')","('P31', 'data_hackers_blog')","('P31', 'data_hackers_podcast')","('P31', 'weekly_newsletter')","('P31', 'slack_channel')","('P31', 'data_hackers_bootcamp')","('P31', 'do_not_know_data_hackers')","('P32', 'prefered_data_hackers_initiative')","('P33', 'telegram_groups')","('P33', 'whatsapp_groups')","('P33', 'youtube_channels')","('P33', 'other_brasilian_blogs')","('P33', 'other_slack_channels')","('P33', 'twitter')","('P33', 'abroad_blogs')","('P33', 'abroad_podcasts')","('P33', 'meetups_and_events')","('P33', 'only_data_hackers')","('P33', 'other')","('P34', 'udacity')","('P34', 'coursera')","('P34', 'udemy')","('P34', 'height')","('P34', 'edx')","('P34', 'data_camp')","('P34', 'data_quest')","('P34', 'kaggle_learn')","('P34', 'online_courses')","('P34', 'other')","('P35', 'data_science_plataforms_preference')","('P35', 'other')","('P36', 'draw_participation')","('D1', 'living_macroregion')","('D2', 'origin_macroregion')","('D3', 'anonymized_degree_area')","('D4', 'anonymized_market_sector')","('D5', 'anonymized_manager_level')","('D6', 'anonymized_role')"
0,v9otv8j9wdvjrv9otvwnn9owhzq54ktv,37.0,Masculino,1,Minas Gerais (MG),1.0,Estudante de Graduação,Empregado (CTL),de 1 a 5,0.0,de R$ 1.001/mês a R$ 2.000/mês,Não tenho experiência na área de dados,Não tive experiência na área de TI/Engenharia ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,Ainda não conhecia o Data Hackers,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,Nunca fiz cursos online,NaN,1.0,Região Sudeste,NaN,Computação / Engenharia de Software / Sistemas...,Outras,NaN,Outras
1,875ul998t0hqcv0871uptwf3oswcfv35,24.0,Feminino,1,São Paulo (SP),1.0,Estudante de Graduação,Empregado (CTL),Acima de 3000,0.0,de R$ 2.001/mês a R$ 3000/mês,Menos de 1 ano,Não tive experiência na área de TI/Engenharia ...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,Ainda não conhecia o Data Hackers,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,NaN,Data Science Academy,0.0,Região Sudeste,NaN,Computação / Engenharia de Software / Sistemas...,Educação,NaN,Data Analyst/Analista de Dados
2,puscuk079vw1pusbb900pzw2xvpxtgdk,26.0,Masculino,1,São Paulo (SP),1.0,Graduação/Bacharelado,Empregado (CTL),Acima de 3000,0.0,de R$ 4.001/mês a R$ 6.000/mês,de 1 a 2 anos,de 6 a 10 anos,1,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,1,0,0,0,Newsletter Semanal,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,Udemy,NaN,1.0,Região Sudeste,NaN,Outras Engenharias,Indústria (Manufatura),NaN,Outras
3,rmel8ewqpbffp2mnfbzermel8eqincov,21.0,Masculino,1,São Paulo (SP),0.0,Estudante de Graduação,Estagiário,de 11 a 50,0.0,de R$ 1.001/mês a R$ 2.000/mês,Menos de 1 ano,de 2 a 3 anos,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,Ainda não conhecia o Data Hackers,0,1,0,0,0,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,Alura,NaN,1.0,Região Sudeste,Região Sudeste,

In [ ]:
# Pré-processamento básico

profmap = {'Desenvolvedor ou Engenheiro de Software': 'Desenvolvedor/Engenheiro de Software', 
                                            'Outras': 'Outras',
                 'Data Scientist/Cientista de Dados': 'Cientista de Dados',
                    'Data Analyst/Analista de Dados': 'Analista de Dados',
              'Business Intelligence/Analista de BI': 'Analista de BI',
                 'Data Engineer/Engenheiro de Dados': 'Engenheiro de Dados',
             'Business Analyst/Analista de Negócios': 'Analista de BI',
               'Analista de Inteligência de Mercado': 'Analista de BI',
                                        'Engenheiro': 'Outras',
                             'Analista de Marketing': 'Outras',
                    'Engenheiro de Machine Learning': 'Cientista de Dados',
               'DBA/Administrador de Banco de Dados': 'Engenheiro de Dados',
                                       'Estatístico': 'Cientista de Dados',
                                        'Economista': 'Outras'}


tammap = { 'de 1 a 5': 'Pequena',
           'de 6 a 10': 'Pequena',
           'de 11 a 50': 'Pequena',
           'de 51 a 100': 'Pequena',
           'de 101 a 500': 'Média',
           'de 501 a 1000': 'Média',
           'de 1001 a 3000': 'Grande',
           'Acima de 3000': 'Grande'}

           
salamap = {    'Menos de R$ 1.000/mês': 1000,
      'de R$ 1.001/mês a R$ 2.000/mês': 1500, 
       'de R$ 2.001/mês a R$ 3000/mês': 2500,
      'de R$ 3.001/mês a R$ 4.000/mês': 3500,
      'de R$ 4.001/mês a R$ 6.000/mês': 5000, 
      'de R$ 6.001/mês a R$ 8.000/mês': 7000,
     'de R$ 8.001/mês a R$ 12.000/mês': 10000,
    'de R$ 12.001/mês a R$ 16.000/mês': 14000,
    'de R$ 16.001/mês a R$ 20.000/mês': 18000,
    'de R$ 20.001/mês a R$ 25.000/mês': 22500, 
              'Acima de R$ 25.001/mês': 25000}
 

def aux(x):
    if x in ['Empregado (CTL)', 'Empreendedor ou Empregado (CNPJ)', 'Estagiário']:
        return x
    else:
        return 'Outros'
 
cortes = [0, 24, 30, 40, 100]
nomes = ['[18,24]', '[25,30]', '[31,40]', '[41,50]']
 

dados['Salário'] = dados["('P16', 'salary_range')"].map(salamap)   
dados['Idade'] = pd.cut(dados["('P1', 'age')"], bins = cortes, labels = nomes) 
dados['Profissão'] = dados["('D6', 'anonymized_role')"].map(profmap) 
dados['Tamanho da Empresa'] = dados["('P12', 'workers_number')"].map(tammap)
dados['Cargo de Gestão'] = dados["('P13', 'manager')"].map({0: 'não', 1: 'sim'})
dados['Experiência em DS'] = dados["('P17', 'time_experience_data_science')"]
dados['Tipo de Trabalho'] = dados["('P10', 'job_situation')"].apply(aux) 
dados['Escolaridade'] = dados["('P8', 'degreee_level')"]
dados['Área de Formação'] = dados["('D3', 'anonymized_degree_area')"]
dados['Setor de Mercado'] = dados["('D4', 'anonymized_market_sector')"] 
dados['Brasil'] = dados["('P3', 'living_in_brasil')"]
dados['Estado'] = dados["('P5', 'living_state')"]
dados['Linguagem Python'] = dados["('P21', 'python')"]
dados['Linguagem R'] = dados["('P21', 'r')"]
dados['Linguagem SQL'] = dados["('P21', 'sql_')"]
dados.head()

In [ ]:
vars = dados.columns[-15:]
dados0 = dados[vars]
dados0.head()

In [6]:
dados0.shape

(1765, 15)

In [7]:
dados0.isnull().mean()

Salário               0.134844
Idade                 0.013598
Profissão             0.291218
Tamanho da Empresa    0.134844
Cargo de Gestão       0.134844
Experiência em DS     0.000000
Tipo de Trabalho      0.000000
Escolaridade          0.000000
Área de Formação      0.019830
Setor de Mercado      0.137677
Brasil                0.000000
Estado                0.190935
Linguagem Python      0.000000
Linguagem R           0.000000
Linguagem SQL         0.000000
dtype: float64

In [26]:
dados0.to_csv('prof-dados-resumido.csv', index = False)

---

In [7]:
!pip install pycaret

In [19]:
from pycaret.regression import setup, create_model, tune_model, evaluate_model, finalize_model, save_model

In [ ]:
# Configuração da Modelagem automática feita pelo PyCaret
# Envolve os procedimentos de validação quanto de pré-processamento

modelagem = setup(data = dados0,
                  target = 'Salário', 
                  ignore_low_variance = True) 

In [21]:
modelo = create_model('rf')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,2640.2716,1.469399e+07,3833.2746,0.2863,0.5277,0.5024
1,2277.6250,1.171561e+07,3422.8076,0.4394,0.5186,0.5407
2,2975.3549,1.791816e+07,4232.9849,0.1140,0.6404,0.6918
3,2474.8843,1.305796e+07,3613.5802,0.4115,0.5202,0.5196
4,2691.3956,1.193333e+07,3454.4648,0.1027,0.5548,0.5932
5,2543.6511,1.390961e+07,3729.5593,0.3235,0.6049,0.6686
6,2619.8808,1.528744e+07,3909.9160,0.4379,0.5453,0.5367
7,2511.8629,1.392589e+07,3731.7404,0.3949,0.5612,0.5848
8,2722.9439,1.700879e+07,4124.1709,0.4904,0.5567,0.5641
9,2484.9969,1.240630e+07,3522.2580,0.3880,0.5157,0.4776


In [22]:
modelo_tunado = tune_model(modelo, optimize = 'MAE', n_iter = 10)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,2571.0359,1.361750e+07,3690.1900,0.3386,0.5081,0.4678
1,2433.3912,1.204735e+07,3470.9288,0.4236,0.5141,0.5260
2,2767.7083,1.597206e+07,3996.5059,0.2102,0.6058,0.6452
3,2353.1829,1.231610e+07,3509.4308,0.4449,0.4829,0.4440
4,2355.3154,9.368623e+06,3060.8207,0.2956,0.5159,0.5139
5,2265.9755,1.171837e+07,3423.2110,0.4301,0.5462,0.5496
6,2585.5432,1.623677e+07,4029.4871,0.4030,0.5305,0.4908
7,2388.8143,1.412661e+07,3758.5381,0.3862,0.5474,0.5388
8,2735.0759,1.868506e+07,4322.6214,0.4402,0.5424,0.5163
9,2392.8154,1.268109e+07,3561.0512,0.3745,0.5068,0.4391


In [23]:
evaluate_model(modelo)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [24]:
modelo_final = finalize_model(modelo_tunado)

In [ ]:
save_model(modelo_final, 'modelo-para-previsao-de-salario')